# API Data Extractor
Last code chunk is what worked. I'll clean up this notebook this week and refine the data extraction process.

In [17]:
import requests
import json
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import time
import certifi




In [6]:
import requests
from bs4 import BeautifulSoup

def fetch_sp500_tickers():
    """
    Fetch the list of S&P 500 ticker symbols from Wikipedia.

    Returns:
    list: A list of S&P 500 ticker symbols.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data from Wikipedia: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []

    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip()
        tickers.append(ticker)

    return tickers

# Example usage
sp500_tickers = fetch_sp500_tickers()
print(f"Fetched {len(sp500_tickers)} S&P 500 ticker symbols.")
print(sp500_tickers)

# Save tickers to a file for later use
with open('sp500_tickers.txt', 'w') as f:
    for ticker in sp500_tickers:
        f.write(ticker + '\n')

Fetched 503 S&P 500 ticker symbols.
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'CO

In [3]:
def fetch_stock_news(api_key, ticker, limit=50):
    """
    Fetch stock news from Financial Modeling Prep API.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    ticker (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
    limit (int): Number of news articles to fetch. Default is 50.

    Returns:
    list: A list of news articles.
    """
    url = f"https://financialmodelingprep.com/api/v3/stock_news?tickers={ticker}&limit={limit}&apikey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        news_data = response.json()
        return news_data
    else:
        print(f"Failed to fetch news: {response.status_code}")
        return []

def save_news_to_file(news_data, filename):
    """
    Save news data to a JSON file.

    Parameters:
    news_data (list): The list of news articles to save.
    filename (str): The name of the file to save the news articles.
    """
    with open(filename, 'w') as file:
        json.dump(news_data, file, indent=4)
    print(f"News data saved to {filename}")


In [10]:
# Example usage
api_key = '28a3eef526c43ab5888ab02222aada18'
ticker = 'AAPL'
news_data = fetch_stock_news(api_key, ticker, limit=200)

if news_data:
    save_news_to_file(news_data, f'{ticker}_news.json')

News data saved to AAPL_news.json


In [ ]:
import time

def fetch_and_save_all_news(api_key, tickers, limit=50):
    """
    Fetch and save news articles for a list of tickers.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    tickers (list): A list of ticker symbols.
    limit (int): Number of news articles to fetch per ticker. Default is 50.
    """
    for ticker in tickers:
        print(f"Fetching news for {ticker}...")
        news_data = fetch_stock_news(api_key, ticker, limit)
        if news_data:
            save_news_to_file(news_data, f'{ticker}_news.json')
        time.sleep(1)  # Sleep to avoid hitting API rate limits

# Example usage
api_key = 'your_api_key_here'
sp500_tickers = fetch_sp500_tickers()

if sp500_tickers:
    fetch_and_save_all_news(api_key, sp500_tickers)

In [14]:
import xml.etree.ElementTree as ET

def fetch_stock_news_sentiments(api_key, ticker, limit=50):
    """
    Fetch stock news sentiments from Financial Modeling Prep RSS feed API.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    ticker (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
    limit (int): Number of news articles to fetch. Default is 50.

    Returns:
    list: A list of news sentiments.
    """
    url = f"https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?page={limit}&apikey={api_key}"

    #url = f"https://financialmodelingprep.com/api/v4/rss_feed?symbol={ticker}&apikey={api_key}&limit={limit}"
    response = requests.get(url)
    
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        news_sentiments = []
        
        for item in root.findall('./channel/item'):
            news = {
                'title': item.find('title').text,
                'link': item.find('link').text,
                'pubDate': item.find('pubDate').text,
                'description': item.find('description').text,
                'sentiment': item.find('sentiment').text
            }
            news_sentiments.append(news)
        
        return news_sentiments
    else:
        print(f"Failed to fetch news sentiments: {response.status_code}")
        return []

def save_news_sentiments_to_file(news_sentiments, filename):
    """
    Save news sentiments to a JSON file.

    Parameters:
    news_sentiments (list): The list of news sentiments to save.
    filename (str): The name of the file to save the news sentiments.
    """
    with open(filename, 'w') as file:
        json.dump(news_sentiments, file, indent=4)
    print(f"News sentiments saved to {filename}")



In [15]:
# Example usage
ticker = 'AAPL'
news_sentiments = fetch_stock_news_sentiments(api_key, ticker)

if news_sentiments:
    save_news_sentiments_to_file(news_sentiments, f'{ticker}_news_sentiments.json')

ParseError: syntax error: line 1, column 0 (<string>)

In [ ]:
def fetch_and_save_all_sentiment_news(api_key, tickers, limit=50):
    """
    Fetch and save news articles for a list of tickers.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    tickers (list): A list of ticker symbols.
    limit (int): Number of news articles to fetch per ticker. Default is 50.
    """
    for ticker in tickers:
        print(f"Fetching news for {ticker}...")
        news_sentiments = fetch_stock_news_sentiments(api_key, ticker, limit)
        if news_sentiments:
            save_news_sentiments_to_file(news_sentiments, f'{ticker}_news_sentiments.json')
        time.sleep(1)  # Sleep to avoid hitting API rate limits



In [ ]:
# Example usage
sp500_tickers_200 = sp500_tickers[:200]

if sp500_tickers:
    fetch_and_save_all_news(api_key, sp500_tickers, limit=100)

In [ ]:
def fetch_stock_news(api_key, ticker, limit=50):
    """
    Fetch stock news from Financial Modeling Prep API.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    ticker (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
    limit (int): Number of news articles to fetch. Default is 50.

    Returns:
    list: A list of news articles.
    """
    url = f"https://financialmodelingprep.com/api/v3/stock_news?tickers={ticker}&limit={limit}&apikey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        news_data = response.json()
        return news_data
    else:
        print(f"Failed to fetch news: {response.status_code}")
        return []

In [16]:
#!/usr/bin/env python
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json

def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

url = f"https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?page=1&apikey={api_key}"
print(get_jsonparsed_data(url))


/var/folders/j2/skjc081x2h5cd717r89c0q480000gn/T/ipykernel_14918/438991653.py:13: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


[{'symbol': 'CHRD', 'publishedDate': '2024-06-16T13:28:33.000Z', 'title': 'Top Wall Street analysts suggest these 3 dividend stocks for enhanced returns', 'image': 'https://cdn.snapi.dev/images/v1/y/q/oil44-2481003.jpg', 'site': 'cnbc.com', 'text': "TipRanks' analyst ranking service pinpoints Wall Street's best-performing stocks, including Cisco Systems and Kimberly-Clark", 'url': 'https://www.cnbc.com/2024/06/16/wall-street-analysts-pick-these-3-dividend-stocks-for-enhanced-returns.html', 'sentiment': 'Positive', 'sentimentScore': 0.876}, {'symbol': 'KKOYF', 'publishedDate': '2024-06-16T13:19:08.000Z', 'title': 'Kesko: Long-Term Upside In Trading And Groceries', 'image': 'https://cdn.snapi.dev/images/v1/n/t/stores-grocery18-2481008.jpg', 'site': 'seekingalpha.com', 'text': 'Despite some volatility in earnings, Kesko has a history of meeting estimates and offers potential for growth and capital appreciation. Kesko is a solid long-term investment with good upside potential, offering a 6

# This is the code for Pulling articles and sentiment analysis from the api 

In [5]:
import certifi
import json
from urllib.request import urlopen
import time
import requests
from bs4 import BeautifulSoup

def get_jsonparsed_data(url):
    """
    Fetch JSON data from the given URL.

    Parameters:
    url (str): The URL to fetch data from.

    Returns:
    dict: The parsed JSON data.
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

def fetch_stock_news_sentiments(api_key, ticker, page=0):
    """
    Fetch stock news sentiments from Financial Modeling Prep API.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    ticker (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
    page (int): Page number for pagination. Default is 0.

    Returns:
    dict: The parsed JSON data containing news sentiments.
    """
    url = f"https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?symbol={ticker}&page={page}&apikey={api_key}"
    return get_jsonparsed_data(url)

def save_news_sentiments_to_file(all_news_sentiments, filename):
    """
    Save all news sentiments to a single JSON file.

    Parameters:
    all_news_sentiments (list): The list of all news sentiments to save.
    filename (str): The name of the file to save the news sentiments.
    """
    with open(filename, 'w') as file:
        json.dump(all_news_sentiments, file, indent=4)
    print(f"All news sentiments saved to {filename}")

def fetch_and_save_all_news(api_key, tickers, page=0):
    """
    Fetch and save news articles for a list of tickers into a single JSON file.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    tickers (list): A list of ticker symbols.
    page (int): Page number for pagination. Default is 0.
    """
    all_news_sentiments = []
    for ticker in tickers:
        print(f"Fetching news for {ticker}...")
        news_sentiments = fetch_stock_news_sentiments(api_key, ticker, page)
        if news_sentiments:
            all_news_sentiments.extend(news_sentiments)
        time.sleep(1)  # Sleep to avoid hitting API rate limits

    save_news_sentiments_to_file(all_news_sentiments, 'all_news_sentiments.json')

def fetch_sp500_tickers():
    """
    Fetch the list of S&P 500 ticker symbols from Wikipedia.

    Returns:
    list: A list of S&P 500 ticker symbols.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data from Wikipedia: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []

    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip()
        tickers.append(ticker)

    return tickers

# Example usage
api_key = '28a3eef526c43ab5888ab02222aada18'
# fetch top # tickers
#sp500_tickers = sp500_tickers[:5]  
sp500_tickers = fetch_sp500_tickers()

if sp500_tickers:
    fetch_and_save_all_news(api_key, sp500_tickers)

Fetching news for MMM...


/var/folders/j2/skjc081x2h5cd717r89c0q480000gn/T/ipykernel_28818/43353743.py:18: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


Fetching news for AOS...
Fetching news for ABT...
Fetching news for ABBV...
Fetching news for ACN...
Fetching news for ADBE...
Fetching news for AMD...
Fetching news for AES...
Fetching news for AFL...
Fetching news for A...
Fetching news for APD...
Fetching news for ABNB...
Fetching news for AKAM...
Fetching news for ALB...
Fetching news for ARE...
Fetching news for ALGN...
Fetching news for ALLE...
Fetching news for LNT...
Fetching news for ALL...
Fetching news for GOOGL...
Fetching news for GOOG...
Fetching news for MO...
Fetching news for AMZN...
Fetching news for AMCR...
Fetching news for AEE...
Fetching news for AAL...
Fetching news for AEP...
Fetching news for AXP...
Fetching news for AIG...
Fetching news for AMT...
Fetching news for AWK...
Fetching news for AMP...
Fetching news for AME...
Fetching news for AMGN...
Fetching news for APH...
Fetching news for ADI...
Fetching news for ANSS...
Fetching news for AON...
Fetching news for APA...
Fetching news for AAPL...
Fetching news 

## Conver to pandas dataframe

In [7]:
import pandas as pd
import json

def json_to_dataframe(json_file):
    """
    Convert JSON file to a pandas DataFrame.

    Parameters:
    json_file (str): The path to the JSON file.

    Returns:
    DataFrame: A pandas DataFrame containing the JSON data.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    # Create a DataFrame from the list of articles
    df = pd.DataFrame(data)
    return df

# Example usage
json_file = 'all_news_sentiments.json'
df = json_to_dataframe(json_file)
print(df.head())  # Print the first few rows of the DataFrame

  symbol             publishedDate  \
0    DRI  2024-06-17T00:19:03.000Z   
1   ADSK  2024-06-17T00:11:18.000Z   
2   ADSK  2024-06-17T00:00:00.000Z   
3    HCM  2024-06-17T00:00:00.000Z   
4    QQQ  2024-06-16T23:41:08.000Z   

                                               title  \
0         The 3 Best Food Stocks to Buy in June 2024   
1  Activist Starboard Value takes about $500 mln ...   
2     Activist Starboard Value Has Stake in Autodesk   
3  HUTCHMED Highlights Publication of Phase III E...   
4                        Weight Of Evidence Building   

                                               image               site  \
0  https://cdn.snapi.dev/images/v1/n/k/restaurant...  investorplace.com   
1  https://cdn.snapi.dev/images/v1/b/h/software12...        reuters.com   
2  https://cdn.snapi.dev/images/v1/l/l/software11...            wsj.com   
3  https://cdn.snapi.dev/images/v1/a/a/press12-24...  globenewswire.com   
4  https://cdn.snapi.dev/images/v1/g/f/weight-of-...       

In [8]:
df.to_csv('all_news_sentiments.csv', index=False)

In [9]:
df.to_parquet('all_news_sentiments.parquet', index=False)